In [1]:
import pandas as pd
import glob

In [2]:
def process_file(file_path):
    #Read the specific column (C) and rows (3 to 5), with no header's 
    head = pd.read_excel(file_path, usecols="C", nrows=5, header=None)
    #Remove the head. iloc, (double point and two spaces) from each cell 
    head.iloc[:, 0] = head.iloc[:, 0].str.replace(": ", "", regex=False)
    
    station_id = head.iloc[0, 0] #ID as text station name head.lloc(1, 0) Nome as text
    station_name = head.iloc[1, 0] #ID as text station name head.lloc(1, 0) Nome as text
    latitude = float(head. iloc [2, 0]) #Latitude as a number
    longitude = float (head.iloc[3, 0]) 
    elevation = float (head. iloc[4, 0]) 
    
    #Read data starting from now 9 without Limiting the number of rous 
    data = pd.read_excel(file_path, usecols="A:K", skiprows=8)
    #Find the index of the row containing "Keterangan:"
    keterangan_index = data [data.iloc[:, 0].str.contains("Keterangan :", na=False)].index
    if not keterangan_index.empty: 
        last_row_index = keterangan_index[0] - 4 # rows before the "Keterangan row 
        relevant_data = data.loc[:last_row_index]
    else:
        relevant_data = data #If "Keterangan Es not found, use all data
    #Transpose the data
    data_transposed = relevant_data.T
    #Extract the header from now 1 (Sodes & after transposition)
    headers = data_transposed.iloc[0].values 
    #Extract the favy data fron row 4 (Index 3 after transposition) 
    tavg_data = data_transposed.iloc[3].values
    #create a dataFrame
    tavg_df = pd.DataFrame([tavg_data], columns=headers)
    
    metadata = {
        'station_id': [station_id],
        'station_name': [station_name],
        'latitude': [latitude],
        'longitude': [longitude],
        'elevation': [elevation]
    }
    metadata_df = pd.DataFrame(metadata)
    #Combine metadata with Tavg data
    combined_data = pd.concat([metadata_df, tavg_df], axis=1)
    
    return combined_data
#Example usage with file path
#combined data process file('path_to_your_file.xlsx')

In [5]:
#Get a List of all relevant files
file_pattern = 'data_kalimantan/laporan_iklim_harian*.xlsx'
files = glob.glob(file_pattern)
#Initialize an empty List to hold DataFrames
all_data = []

# Process each file and append the DataFrame to the list
for file in files:
    df = process_file(file)
    all_data.append(df)

In [6]:
final_combined_data = pd.concat(all_data, ignore_index=True)

print(final_combined_data)

   station_id                                       station_name  latitude  \
0       96615                  Stasiun Meteorologi Rahadi Oesman  -1.80000   
1       96645                       Stasiun Meteorologi Iskandar  -2.73000   
2       96651                        Stasiun Meteorologi H. Asan  -2.55000   
3       96653                         Stasiun Meteorologi Sanggu  -1.67000   
4       96595                       Stasiun Meteorologi Beringin  -0.56000   
5       96655                   Stasiun Meteorologi Tjilik Riwut  -2.22000   
6       96529                     Stasiun Meteorologi Kalimarau    2.14562   
7       96633   Stasiun Meteorologi Sultan Aji Muhammad Sulai...  -1.26000   
8       96635                       Stasiun Geofisika Balikpapan  -1.25530   
9       96607   Stasiun Meteorologi Aji Pangeran Tumenggung P...  -0.48000   
10      96525                Stasiun Meteorologi Tanjung Harapan   2.50000   
11      96559                       Stasiun Meteorologi Tebelian

In [7]:
output_file = 'data_kalimantan/hasil.xlsx'
final_combined_data.to_excel(output_file, index=False)